<a href="https://colab.research.google.com/github/astute-ai-gen-ai/harshith/blob/main/u2net_backgrounderase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install OpenCV
!pip install opencv-python-headless

# Install NumPy
!pip install numpy

# Install ONNX
!pip install onnx

# Install ONNX Runtime
!pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.0 MB/s eta 0:00:00


In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=3eaa4c548ecf6e23d06e1c930f7109e5bb8c988538475a9a4a08602b22790c51
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
!wget -O u2net.onnx https://huggingface.co/spaces/xiongjie/u2net_rgba/resolve/main/u2net.onnx


--2024-09-07 16:15:18--  https://huggingface.co/spaces/xiongjie/u2net_rgba/resolve/main/u2net.onnx
Resolving huggingface.co (huggingface.co)... 18.154.227.87, 18.154.227.67, 18.154.227.7, ...
Connecting to huggingface.co (huggingface.co)|18.154.227.87|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/spaces/xiongjie/u2net_rgba/309ea1fc458e8f6711efb645da85d1cc2a9cd2aec261d379bba31c0a0ddc78af?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27u2net.onnx%3B+filename%3D%22u2net.onnx%22%3B&Expires=1725984918&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTk4NDkxOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9zcGFjZXMveGlvbmdqaWUvdTJuZXRfcmdiYS8zMDllYTFmYzQ1OGU4ZjY3MTFlZmI2NDVkYTg1ZDFjYzJhOWNkMmFlYzI2MWQzNzliYmEzMWMwYTBkZGM3OGFmP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=M6npPI7JBBC7oo8Vb8oSk2jXTt%7Efs66r61m9GKBnrW3j-JdRMEEpzbe8dReJx4QNHgqBaH1%7E1x

In [ ]:
!ls -lh u2net.onnx


-rw-r--r-- 1 root root 168M Dec 25  2021 u2net.onnx


In [ ]:
import cv2
import numpy as np
import wget

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    v = cv2.equalizeHist(v)
    enhanced_image = cv2.merge((h, s, v))
    enhanced_image = cv2.cvtColor(enhanced_image, cv2.COLOR_HSV2BGR)
    cv2.imwrite('enhanced_product.jpg', enhanced_image)
    return 'enhanced_product.jpg'

def remove_background(image_path):
    image = cv2.imread(image_path)
    segmentation_model = cv2.dnn.readNetFromONNX('/content/u2net.onnx')  # Use absolute path
    blob = cv2.dnn.blobFromImage(image, scalefactor=1.0/255.0, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    segmentation_model.setInput(blob)
    mask = segmentation_model.forward()
    mask = mask.squeeze()
    mask = cv2.resize(mask, (image.shape[1], image.shape[0]))
    mask = (mask > 0.5).astype(np.uint8) * 255
    segmented_image = cv2.bitwise_and(image, image, mask=mask)
    cv2.imwrite('product_without_background.png', segmented_image)
    return mask

def replace_background(mask, new_bg_path):
    product_image = cv2.imread('product_without_background.png')
    background_image = cv2.imread(new_bg_path)

    # Check if background image is loaded correctly
    if background_image is None:
        raise ValueError(f"Could not load the background image from path: {new_bg_path}")

    background_image = cv2.resize(background_image, (product_image.shape[1], product_image.shape[0]))
    mask_inv = cv2.bitwise_not(mask)
    bg_portion = cv2.bitwise_and(background_image, background_image, mask=mask_inv)
    final_image = cv2.add(bg_portion, product_image)
    cv2.imwrite('final_product_image.jpg', final_image)


def upscale_image(image_path):
    image = cv2.imread(image_path)
    upscaled_image = cv2.resize(image, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('enhanced_final_product_image.jpg', upscaled_image)



In [ ]:
# Run the full pipeline
# enhanced_image_path = preprocess_image('/content/product.jpg')
# mask = remove_background(enhanced_image_path)
mask = remove_background('/content/product.jpg')

replace_background(mask, '/content/background.jpg')
upscale_image('final_product_image.jpg')
